In [3]:
import numpy as np

def inner_product(vec1, vec2):
    """Compute the inner product of two complex vectors."""
    return np.vdot(vec1, vec2)

def encode_to_qubit(arr):
    """
    Direct amplitude encoding:
    Groups pairs of reals into complex numbers and normalizes.
    """
    complex_vec = np.array([arr[i] + 1j * arr[i+1] for i in range(0, len(arr), 2)])
    norm = np.linalg.norm(complex_vec)
    return complex_vec / norm if norm != 0 else complex_vec

def rotation_matrix_y(theta):
    """Construct an R_y(theta) rotation matrix."""
    return np.array([
        [np.cos(theta/2), -np.sin(theta/2)],
        [np.sin(theta/2),  np.cos(theta/2)]
    ], dtype=complex)

def rotation_matrix_z(phi):
    """Construct an R_z(phi) phase shift matrix."""
    return np.array([
        [np.exp(-1j * phi/2), 0],
        [0, np.exp(1j * phi/2)]
    ], dtype=complex)

def recursive_state_preparation(psi):
    """
    Recursively prepares a state |psi> (with 2^n amplitudes) from |0...0>
    by splitting the state into left and right halves. At each step,
    we compute the rotation angle theta and relative phase phi so that
      cos(theta/2) = norm(left branch)
      sin(theta/2) = norm(right branch)
    and we force the left branch amplitude to be real and nonnegative.
    
    The unitary on the current qubit is defined as:
      U' = exp(i*phi/2)*R_z(phi)*R_y(theta)
    so that U'|0> = [cos(theta/2), sin(theta/2)*exp(i*phi)].
    """
    # Force global phase so that psi[0] is real and nonnegative.
    if psi[0] != 0:
        psi = psi * np.exp(-1j * np.angle(psi[0]))
    
    n = int(np.log2(len(psi)))
    if n == 0:
        return psi

    half = len(psi) // 2
    norm_left = np.linalg.norm(psi[:half])
    norm_right = np.linalg.norm(psi[half:])
    
    # Compute rotation angle: cos(theta/2)=norm_left, sin(theta/2)=norm_right.
    theta = 2 * np.arctan2(norm_right, norm_left)
    
    # Set the relative phase from the right branch.
    phi = np.angle(psi[half]) if (norm_left > 0 and norm_right > 0) else 0.0

    # Build the corrected single-qubit unitary.
    U = np.exp(1j * phi/2) * (rotation_matrix_z(phi) @ rotation_matrix_y(theta))
    qubit_state = U @ np.array([1, 0], dtype=complex)
    
    # Recursively prepare the left and right branches.
    if norm_left > 0:
        psi_left = psi[:half] / norm_left
        left_state = recursive_state_preparation(psi_left)
    else:
        left_state = np.zeros(half, dtype=complex)
    if norm_right > 0:
        psi_right = psi[half:] / norm_right
        right_state = recursive_state_preparation(psi_right)
    else:
        right_state = np.zeros(half, dtype=complex)
    
    return np.concatenate([qubit_state[0] * left_state,
                           qubit_state[1] * right_state])

def amplitude_encoding(vector):
    """
    Simulates amplitude encoding using only unitary operations
    (as on real quantum hardware) by recursively preparing the state.
    """
    # Convert the real vector into complex amplitudes.
    complex_vec = np.array([vector[i] + 1j * vector[i+1] for i in range(0, len(vector), 2)])
    norm = np.linalg.norm(complex_vec)
    if norm == 0:
        raise ValueError("Vector cannot be zero.")
    target_state = complex_vec / norm

    # Force global phase so that the first amplitude is real and nonnegative.
    if target_state[0] != 0:
        target_state = target_state * np.exp(-1j * np.angle(target_state[0]))
    
    # Prepare the state recursively.
    return recursive_state_preparation(target_state)

# Example vector for encoding (must have an even number of elements).
vector = np.array([0.5, 0.5, 0.5, 0.5, 0.25, 0.75, 0.1, 0.9])

# Encode the state using both methods.
psi_direct = encode_to_qubit(vector)
psi_enc = amplitude_encoding(vector)

# Optionally, force the same global phase on both states.
def fix_global_phase(state):
    if state[0] != 0:
        return state * np.exp(-1j * np.angle(state[0]))
    return state

psi_direct = fix_global_phase(psi_direct)
psi_enc = fix_global_phase(psi_enc)

# Compute the inner product between the two states.
comparison_inner_prod = inner_product(psi_direct, psi_enc)
comparison_magnitude = np.abs(comparison_inner_prod)

print("Encoded Quantum State (Direct Method):", psi_direct)
print("Encoded Quantum State (Enc):", psi_enc)
print("Inner Product Magnitude Between Methods:", comparison_magnitude)

if np.isclose(comparison_magnitude, 1.0, atol=1e-6):
    print("The two methods produce equivalent quantum states (up to a global phase).")
else:
    print("The two methods produce different quantum states.")


Encoded Quantum State (Direct Method): [0.45221563+3.89009787e-17j 0.45221563+3.89009787e-17j
 0.45221563+2.26107816e-01j 0.45221563+3.61772505e-01j]
Encoded Quantum State (Enc): [0.45221563+6.53691246e-34j 0.45221563-7.74608296e-34j
 0.45221563+2.26107816e-01j 0.45221563+3.61772505e-01j]
Inner Product Magnitude Between Methods: 1.0
The two methods produce equivalent quantum states (up to a global phase).
